In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
import pandas as pd

# Training

In [ ]:
training_df = pd.read_csv("Datasets/TrainingWiDS2021.csv")
training_df.sample(3)

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,...,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
91009,91010,181497,30,49.0,21.748106,0,Caucasian,F,170.2,Emergency Department,Accident & Emergency,906,admit,Neuro ICU,0.160417,0,63.0,NaN,121.0,402.02,0,0,NaN,10.0,0.70,NaN,1.0,6.0,0.0,1.0,118.0,100.0,NaN,0,115.0,NaN,NaN,NaN,NaN,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0,41.0,7.34,7.33,183.0,144.0,457.500000,360.000000,43.0,43.0,7.34,7.34,144.0,144.0,360.000000,360.000000,0,0,0,0,0,0,0,0
1006,1007,232177,118,57.0,32.338884,1,Caucasian,F,162.6,Operating Room,Operating Room / Recovery,92,admit,CTICU,0.000694,0,85.5,NaN,202.0,1205.01,1,0,NaN,10.0,0.76,0.44,4.0,6.0,0.0,4.0,261.0,88.0,29.5,0,56.0,48.0,48.0,181.0,7.33,41.0,...,9.8,9.8,29.5,29.5,NaN,NaN,NaN,NaN,191.0,191.0,4.2,4.2,140.0,140.0,6.7,6.7,48.0,48.0,7.33,7.33,181.0,181.0,411.363636,411.363636,48.0,48.0,7.33,7.33,181.0,181.0,411.363636,411.363636,0,0,0,0,0,0,0,1
29069,29070,200913,112,62.0,39.375574,1,Other/Unknown,F,165.0,Operating Room,Operating Room / Recovery,384,admit,CSICU,0.310417,0,107.2,NaN,308.0,1207.01,1,0,NaN,39.0,0.74,0.60,1.0,1.0,0.0,1.0,369.0,116.0,21.9,1,133.0,48.0,48.0,132.0,7.28,6.0,...,10.6,10.6,31.4,31.4,1.3,1.3,NaN,NaN,221.0,221.0,4.1,4.1,136.0,136.0,23.2,23.2,48.0,43.0,7.35,7.28,132.0,95.0,265.000000,220.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,1


In [ ]:
training_df = training_df.drop(columns = ['Unnamed: 0', 'hospital_id'])

In [ ]:
from sklearn.impute import KNNImputer
import numpy as np
def fill_missing_value_for_continuous_atrr(df, name_attribute):
    imputer = KNNImputer(n_neighbors = 5, weights='distance')
    return imputer.fit_transform(df[name_attribute].to_numpy().reshape(-1,1))

In [ ]:
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder
from fancyimpute import KNN
encoder = OrdinalEncoder()
imputer = IterativeImputer(ExtraTreesRegressor())
def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data
def fill_missing_value_for_categorical_atrr(df):
    return pd.DataFrame(np.round(imputer.fit_transform(df.values.reshape(-1,1))))

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


In [ ]:
training_df['ethnicity'] = encode(training_df['ethnicity'])
training_df['gender'] = encode(training_df['gender'])
training_df['hospital_admit_source'] = encode(training_df['hospital_admit_source'])
training_df['icu_admit_source'] = encode(training_df['icu_admit_source'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [ ]:
from sklearn import preprocessing
icu_stay_type_le = preprocessing.LabelEncoder().fit(training_df['icu_stay_type'])
icu_type_le = preprocessing.LabelEncoder().fit(training_df['icu_type'])
# training_df['icu_stay_type'] = preprocessing.LabelEncoder().fit_transform(training_df['icu_stay_type'])
# training_df['icu_type'] = preprocessing.LabelEncoder().fit_transform(training_df['icu_type'])
training_df['icu_stay_type'] = icu_stay_type_le.transform(training_df['icu_stay_type'])
training_df['icu_type'] = icu_type_le.transform(training_df['icu_type'])

# Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.model_selection       import train_test_split
from sklearn.preprocessing         import StandardScaler
from sklearn.preprocessing         import RobustScaler
from sklearn.preprocessing         import QuantileTransformer
from sklearn.preprocessing         import PowerTransformer
from sklearn.preprocessing         import MinMaxScaler
from sklearn.preprocessing         import MaxAbsScaler
from sklearn.pipeline              import Pipeline
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.svm                   import SVC
from sklearn.tree                  import DecisionTreeClassifier
from sklearn.ensemble              import RandomForestClassifier
from sklearn.naive_bayes           import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model          import LogisticRegression
from sklearn.metrics               import mean_squared_error
from sklearn.model_selection       import cross_val_score
from scipy.stats import uniform, randint
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

from sklearn.metrics import accuracy_score

In [ ]:
training_df['ethnicity'] = training_df['ethnicity'].astype('float64')
training_df['gender'] = training_df['gender'].astype("float64")
training_df['hospital_admit_source'] = training_df['hospital_admit_source'].astype("float64")
training_df['icu_admit_source'] = training_df['icu_admit_source'].astype('float64')

In [ ]:
X = training_df.iloc[:,:-1]
y = training_df.iloc[:,-1]

In [ ]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, eval_metric="aucpr", colsample_bytree = 0.3, learning_rate = 0.042260584879943656,
                max_depth = 20, alpha = 10, n_estimators = 150)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size= 0.2, random_state=42)

In [ ]:
robust = RobustScaler().fit(X_train)
X_train = robust.transform(X_train)
X_test = robust.transform(X_test)

In [ ]:
xgb_model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_test, y_test)])

y_pred = xgb_model.predict(X_test)

accuracy_score(y_test, y_pred)

[0]	validation_0-aucpr:0.366462
Will train until validation_0-aucpr hasn't improved in 5 rounds.
[1]	validation_0-aucpr:0.369417
[2]	validation_0-aucpr:0.431801
[3]	validation_0-aucpr:0.437576
[4]	validation_0-aucpr:0.448448
[5]	validation_0-aucpr:0.46409
[6]	validation_0-aucpr:0.488142
[7]	validation_0-aucpr:0.504893
[8]	validation_0-aucpr:0.507277
[9]	validation_0-aucpr:0.508001
[10]	validation_0-aucpr:0.525387
[11]	validation_0-aucpr:0.527659
[12]	validation_0-aucpr:0.536155
[13]	validation_0-aucpr:0.537963
[14]	validation_0-aucpr:0.546558
[15]	validation_0-aucpr:0.550307
[16]	validation_0-aucpr:0.556031
[17]	validation_0-aucpr:0.559749
[18]	validation_0-aucpr:0.55889
[19]	validation_0-aucpr:0.560275
[20]	validation_0-aucpr:0.561263
[21]	validation_0-aucpr:0.561955
[22]	validation_0-aucpr:0.56814
[23]	validation_0-aucpr:0.569503
[24]	validation_0-aucpr:0.569515
[25]	validation_0-aucpr:0.569059
[26]	validation_0-aucpr:0.571196
[27]	validation_0-aucpr:0.571982
[28]	validation_0-aucpr:

0.8288260602335586

In [ ]:
# Parameter Tuning
model = xgb.XGBClassifier()
param_dist = {"max_depth": [10, 50],
              "min_child_weight" : [1, 6],
              "n_estimators": [200],
              "learning_rate": [0.05, 0.1, 0.16],}
def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))
    
# grid_search = GridSearchCV(model, param_grid=param_dist, cv = 2, 
#                                    verbose=10, n_jobs=-1)
# grid_search.fit(X_train, y_train)

# grid_search.best_estimator_

model = xgb.XGBClassifier(max_depth = 35, min_child_weight=1,  n_estimators=500,\
                         verbose=1,learning_rate=0.08, eval_metric="aucpr", objective="binary:logistic",\
                          random_state=42, colsample_bytree = 0.8, alpha = 10, gamma = 0.25)
model.fit(X_train,y_train, early_stopping_rounds=5, eval_set=[(X_train, y_train), (X_test, y_test)])

accuracy_score(y_test, y_pred)

[0]	validation_0-aucpr:0.914747	validation_1-aucpr:0.461166
Multiple eval metrics have been passed: 'validation_1-aucpr' will be used for early stopping.

Will train until validation_1-aucpr hasn't improved in 5 rounds.
[1]	validation_0-aucpr:0.957197	validation_1-aucpr:0.49268
[2]	validation_0-aucpr:0.967895	validation_1-aucpr:0.520964
[3]	validation_0-aucpr:0.972823	validation_1-aucpr:0.529104
[4]	validation_0-aucpr:0.977554	validation_1-aucpr:0.53794
[5]	validation_0-aucpr:0.981023	validation_1-aucpr:0.546415
[6]	validation_0-aucpr:0.983775	validation_1-aucpr:0.55244
[7]	validation_0-aucpr:0.986082	validation_1-aucpr:0.557399
[8]	validation_0-aucpr:0.987806	validation_1-aucpr:0.56201
[9]	validation_0-aucpr:0.989808	validation_1-aucpr:0.565924
[10]	validation_0-aucpr:0.991313	validation_1-aucpr:0.570303
[11]	validation_0-aucpr:0.992371	validation_1-aucpr:0.572991
[12]	validation_0-aucpr:0.993315	validation_1-aucpr:0.575556
[13]	validation_0-aucpr:0.994164	validation_1-aucpr:0.578755


0.8288260602335586

# Test

In [ ]:
test_df = pd.read_csv("Datasets/UnlabeledWiDS2021.csv")
test_df.sample(3)

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,icu_admit_source,icu_id,icu_stay_type,icu_type,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,...,h1_hco3_min,h1_hemaglobin_max,h1_hemaglobin_min,h1_hematocrit_max,h1_hematocrit_min,h1_inr_max,h1_inr_min,h1_lactate_max,h1_lactate_min,h1_platelets_max,h1_platelets_min,h1_potassium_max,h1_potassium_min,h1_sodium_max,h1_sodium_min,h1_wbc_max,h1_wbc_min,d1_arterial_pco2_max,d1_arterial_pco2_min,d1_arterial_ph_max,d1_arterial_ph_min,d1_arterial_po2_max,d1_arterial_po2_min,d1_pao2fio2ratio_max,d1_pao2fio2ratio_min,h1_arterial_pco2_max,h1_arterial_pco2_min,h1_arterial_ph_max,h1_arterial_ph_min,h1_arterial_po2_max,h1_arterial_po2_min,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis
2441,2442,141604,10128,63,27.8,0,Caucasian,M,172.7,Floor,Floor,436,admit,Med-Surg ICU,3.608333,0,82.8,2.6,108.0,203.01,0,0,7.7,53.0,2.95,1.0,1.0,5.0,0.0,1.0,110.0,112.0,24.9,0,183.0,35.0,35.0,47.0,7.420,9.0,...,20.0,8.2,8.2,25.1,25.1,2.30,2.00,NaN,NaN,247.0,247.0,4.1,4.1,124.0,124.0,20.80,20.80,38.0,34.0,7.42,7.300,89.0,41.0,89.0,47.0,38.0,34.0,7.380,7.300,57.0,41.0,57.0,57.0,0,0,1,0,0,0,0
9138,9139,144856,10059,47,35.7,0,Caucasian,F,157.5,Emergency Department,Accident & Emergency,1025,admit,Med-Surg ICU,0.093750,0,88.5,NaN,302.0,109.04,0,0,NaN,7.0,0.56,NaN,4.0,6.0,0.0,5.0,84.0,104.0,35.5,0,61.0,NaN,NaN,NaN,NaN,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0
2912,2913,144228,10071,75,37.9,1,Caucasian,M,175.0,Operating Room,Operating Room / Recovery,491,admit,Med-Surg ICU,0.400694,0,116.0,NaN,203.0,1206.03,1,0,NaN,21.0,0.80,0.7,4.0,6.0,0.0,5.0,215.0,124.0,29.0,1,60.0,42.5,42.5,102.0,7.399,25.0,...,24.0,9.6,9.5,29.8,29.0,1.64,1.64,NaN,NaN,132.0,132.0,4.3,4.1,142.0,141.0,13.12,13.12,43.3,39.7,7.43,7.384,179.0,72.7,223.8,145.7,39.7,39.7,7.408,7.408,153.0,153.0,NaN,NaN,0,0,0,0,0,0,0


In [ ]:
test_df = test_df.drop(columns = ['Unnamed: 0', 'hospital_id'])

In [ ]:
test_df['ethnicity'] = encode(test_df['ethnicity'])
test_df['gender'] = encode(test_df['gender'])
test_df['hospital_admit_source'] = encode(test_df['hospital_admit_source'])
test_df['icu_admit_source'] = encode(test_df['icu_admit_source'])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setit

In [ ]:
from sklearn import preprocessing
# test_df['icu_stay_type'] = preprocessing.LabelEncoder().fit_transform(test_df['icu_stay_type'])
# test_df['icu_type'] = preprocessing.LabelEncoder().fit_transform(test_df['icu_type'])
test_df['icu_stay_type'] = icu_stay_type_le.transform(test_df['icu_stay_type'])
test_df['icu_type'] = icu_type_le.transform(test_df['icu_type'])

In [ ]:
test_df['ethnicity'] = test_df['ethnicity'].astype('float64')
test_df['gender'] = test_df['gender'].astype("float64")
test_df['hospital_admit_source'] = test_df['hospital_admit_source'].astype("float64")
test_df['icu_admit_source'] = test_df['icu_admit_source'].astype('float64')

In [ ]:
probabilities = xgb_model.predict_proba(test_df)

In [ ]:
probabilities = model.predict_proba(test_df)

In [ ]:
probabilities = model.predict_proba(robust.transform(test_df))

In [ ]:
probabilities = xgb_model.predict_proba(robust.transform(test_df))

In [ ]:
result = pd.DataFrame()

In [ ]:
result['encounter_id'] = test_df['encounter_id']
result['diabetes_mellitus'] = probabilities[:, 1]

In [ ]:
result['diabetes_mellitus'] = probabilities[:, 1]

In [ ]:
result.head(10)

,encounter_id,diabetes_mellitus
0,144740,0.026277
1,141990,0.066868
2,142038,0.057907
3,138628,0.012932
4,141682,0.370802
5,139096,0.009144
6,142994,0.886868
7,141954,0.012831
8,135344,0.901361
9,142552,0.893350


In [ ]:
model_name = 'XGBOOST'

# Submission

In [ ]:
result.to_csv(model_name+".csv", index = False)